# Active Learning Experiment Notebook

##### Things we need to be able to access here

inputs: im_dir


discriminator (want to see disc_model.model.plot_distribution(img_dir) sorted image_id along x axis and y axis is disc score, disc_model.model.show_disc(filepaths), disc_model.model.verts = dict(where vertical bars should go each time we call update_model)) 
segmenter (want to have a seg_model.segmenter.show_segmentations(filepaths), seg_model.segmenter.verts = dict(where vertical bars should go) ) 
if using autooracle, want visualization of what the oracle is seeing/saying
oracle_results, oracle_thresholds
training curves (make sure to maintain a history from initial training)
TODO: convergence criterion? 

### Import Statements

In [1]:
#Python Library imports

#Backend py file imports
import floodfill
import disc_model
import oracle
import gen_model

### Seed all Random Generators

In [ ]:
random_seed_number = 44


In [ ]:
## TODO: Where to put these
torch.manual_seed(random_seed_number)
torch.cuda.manual_seed(random_seed_number)
np.random.seed(random_seed_number)
random.seed(random_seed_number)

torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True

### Run ID Setup

In [ ]:
#run_id has format of "yy_mm_dd_count", where count is the current run on the day (a,b,etc)
run_id = "23_03_30_a" 

In [ ]:
#users_name tells us who is working on the notebook (vaibhav/alina/julia)
users_name = input("what is your name: ")
print(f"Your name is: {users_name}.")

## Active Learning Stage

### Load Segmenter

### Initialize Segmenter

### Set Image Directories

In [ ]:
Segmenter initial training train/valid images
Discriminator initial training images

Images to be segmented by AL system 

duke_valid
duke_testA
duke_testB





In [ ]:
im_dir = "" #im_dir is the directory where oracle pulls images

In [ ]:
dataloader = get_DataLoader(im_dir,32,2) #Generates dataloader from im_dir. Takes in batch_size and num_workers

#Initializes oracle results dict and thresholds dict
oracle_results = {}
oracle_results_thresholds = {}

### Initial Discriminator Training

In [ ]:
model,loss_tracker,criterion,optimizer = initialize_and_train_model(dataloader, epochs=5) #default batch_size and epochs
plt.plot(loss_tracker) #plot graph

In [ ]:
#Gets the patient scores based on initial trained model. Patient scores is how "good" the model thinks the segmentation is
all_patient_scores = []

patient_scores = get_patient_scores(model,dataloader)  #patient_scores is a dictionary of patient->score
all_patient_scores.append(patient_scores)

### Oracle Querying

In [ ]:
#Queries the oracle. Relevant arguments: query_method and query_number
#Query methods: best, worst, percentile=0.x, uniform.
#Put 1 if correct, 0 if impossible, new threshold if new threshold will help.

new_oracle_results, new_oracle_results_thresholds = query_oracle(oracle_results,oracle_results_thresholds,
                                                                 patient_scores,im_dir,query_method="best",
                                                                 query_number=13)
oracle_results, oracle_results_thresholds = new_oracle_results, new_oracle_results_thresholds

### Updating the Discriminator

In [ ]:
#Update patient scores and add to all_patient_scores array
patient_scores = get_patient_scores(model,dataloader)
all_patient_scores.append(patient_scores)

In [ ]:
#Update the active learning classifier with learned data
#TODO: track model loss somehow along with patient_scores (KEEP TODO AND ADDRESS)
for i in range(3):
    model = model_update(model,dataloader,oracle_results,criterion,optimizer,num_epochs=1)

    patient_scores = get_patient_scores(model,dataloader)
    all_patient_scores.append(patient_scores)

**Go Back to Oracle Querying Heading if you want to keep querying images**

### Update Segmenter

In [1]:
## Code

### Plotting Active Learning Metrics

In [ ]:
#Prints out metrics for all the patient scores from each update.
for i in all_patient_scores:
    print(oracle.calculate_dispersion_metric(i,oracle_results))

In [ ]:
#Plot the disperson metric
j = []
for i in all_patient_scores:
    j.append(oracle.calculate_dispersion_metric(i,oracle_results))
    
plt.plot(j)

In [ ]:
print("Length of patient scores: " + str(len(patient_scores)))
print("Length of oracle results: " + str(len(oracle_results)))

scores = []
for key in patient_scores.keys():
    scores.append(patient_scores[key])
plt.plot(scores)

ones = 0
for i in oracle_results.keys():
    if oracle_results[i]==1:
        ones+=1
print("Number of ones in oracle results: ", ones)

In [58]:
im_dir = "/usr/xtmp/vs196/mammoproj/Data/manualfa/train/" #manually labelled training segmentations
manual_fa_train_dir = "/usr/xtmp/vs196/mammoproj/Data/manualfa/train/"
model_save_dir = "/usr/xtmp/vs196/mammoproj/Code/SavedModels/ControlALUNet/0726/"

if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)